# Testing fs8

In [1]:
Threads.nthreads()

1

In [2]:
using LinearAlgebra
using Turing
using LimberJack
using GaussianProcess
using PythonCall
using NPZ
using BenchmarkTools;

In [3]:
np = pyimport("numpy");

In [4]:
meta = np.load("../data/DESY1/gcgc_gcwl_wlwl_meta.npz")
tracers_names = pyconvert(Vector{String}, meta["tracers"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"])
pairs_ids = pyconvert(Vector{Vector{Int}}, meta["pairs_ids"])
idx = pyconvert(Vector{Int}, meta["idx"])
cls_data = pyconvert(Vector{Float64}, meta["cls"])
cls_cov = pyconvert(Matrix{Float64}, meta["cov"])
cls_inv_cov = pyconvert(Matrix{Float64}, meta["inv_cov"]);

In [5]:
files = npzread("../data/DESY1/gcgc_gcwl_wlwl_files.npz");

In [6]:
fs8_meta = npzread("../data/fs8s/fs8s.npz")
fs8_zs = fs8_meta["z"]
fs8_data = fs8_meta["data"]
fs8_cov = fs8_meta["cov"]

11×11 Matrix{Float64}:
 0.00203355   0.000811829  0.000264615  …  0.0     0.0     0.0       0.0
 0.000811829  0.00142289   0.000662824     0.0     0.0     0.0       0.0
 0.000264615  0.000662824  0.00118576      0.0     0.0     0.0       0.0
 0.0          0.0          0.0             0.0     0.0     0.0       0.0
 0.0          0.0          0.0             0.0     0.0     0.0       0.0
 0.0          0.0          0.0          …  0.0     0.0     0.0       0.0
 0.0          0.0          0.0             0.0     0.0     0.0       0.0
 0.0          0.0          0.0             0.0144  0.0     0.0       0.0
 0.0          0.0          0.0             0.0     0.0121  0.0       0.0
 0.0          0.0          0.0             0.0     0.0     0.003025  0.0
 0.0          0.0          0.0          …  0.0     0.0     0.0       0.013456

In [7]:
cov_tot = zeros(Float64, length(fs8_data)+length(cls_data), length(fs8_data)+length(cls_data))
cov_tot[1:length(fs8_data), 1:length(fs8_data)] = fs8_cov
cov_tot[length(fs8_data)+1:(length(fs8_data)+length(cls_data)),
        length(fs8_data)+1:(length(fs8_data)+length(cls_data))] = cls_cov
data_vector = [fs8_data ; cls_data];

In [8]:
fid_cosmo = Cosmology()
N = 100
latent_x = Vector(0:0.3:3)
x = Vector(range(0., stop=3., length=N));

In [9]:
pars = [4.426868e-02,     2.093138e-01,     8.963611e-01,     8.495440e-01,
         1.343888e+00,    1.639047e+00,      1.597174e+00,     1.944583e+00,     2.007245e+00,
        -4.679383e-03,   -2.839996e-03,      1.771571e-03,     1.197051e-03,    -5.199799e-03,
         2.389208e-01,   -6.435288e-01, 
         1.802722e-03,   -5.508994e-03,     1.952514e-02,    -1.117726e-03,
        -1.744083e-02,    6.777779e-03,    -1.097939e-03,    -4.912315e-03,
         8.536883e-01,    2.535825e-01];

In [10]:
nuisances = Dict("DESgc__0_0_b" => pars[5],
                 "DESgc__1_0_b" => pars[6],
                 "DESgc__2_0_b" => pars[7],
                 "DESgc__3_0_b" => pars[8],
                 "DESgc__4_0_b" => pars[9],
                 "DESgc__0_0_dz" => pars[10],
                 "DESgc__1_0_dz" => pars[11],
                 "DESgc__2_0_dz" => pars[12],
                 "DESgc__3_0_dz" => pars[13],
                 "DESgc__4_0_dz" => pars[14],

                 "A_IA" => pars[15],
                 "alpha_IA" => pars[16],

                 "DESwl__0_e_dz" => pars[21],
                 "DESwl__1_e_dz" => pars[22],
                 "DESwl__2_e_dz" => pars[23],
                 "DESwl__3_e_dz" => pars[24],
                 "DESwl__0_e_m" => pars[17],
                 "DESwl__1_e_m" => pars[18],
                 "DESwl__2_e_m" => pars[19],
                 "DESwl__3_e_m" => pars[20])

Dict{String, Float64} with 20 entries:
  "DESwl__2_e_dz" => -0.00109794
  "DESwl__3_e_m"  => -0.00111773
  "A_IA"          => 0.238921
  "DESwl__0_e_m"  => 0.00180272
  "DESwl__3_e_dz" => -0.00491232
  "DESgc__3_0_dz" => 0.00119705
  "DESgc__1_0_b"  => 1.63905
  "DESgc__2_0_b"  => 1.59717
  "DESwl__1_e_dz" => 0.00677778
  "DESwl__1_e_m"  => -0.00550899
  "DESgc__4_0_dz" => -0.0051998
  "DESgc__3_0_b"  => 1.94458
  "DESwl__2_e_m"  => 0.0195251
  "DESgc__1_0_dz" => -0.00284
  "DESgc__0_0_dz" => -0.00467938
  "DESgc__4_0_b"  => 2.00725
  "alpha_IA"      => -0.643529
  "DESgc__0_0_b"  => 1.34389
  "DESwl__0_e_dz" => -0.0174408
  "DESgc__2_0_dz" => 0.00177157

## Tests

In [11]:
function TTheory(cosmology::Cosmology,
                tracers_names, pairs,
                idx, files;
                Nuisances=Dict())

    ntracers = length(tracers_names)
    tracers = []
    
    nui_type =  eltype(valtype(Nuisances))
    if !(nui_type <: Float64) & (nui_type != Any)
        if nui_type != Real
            cosmology.settings.cosmo_type = nui_type
        end
    end
    
    tracers =  Dict{String}{LimberJack.Tracer}()
    
    @inbounds for name in tracers_names
        n = length(name)
        t_type = name[n:n]
        if t_type == "0"
            nzs = files[string("nz_", name)]
            nzs = [nzs[i,:] for i in 1:size(nzs,1)]
            zs_mean, nz_mean = nzs[1], nzs[2]

            b = get(Nuisances, string(name, "_", "b"), 1.0)
            nz = get(Nuisances, string(name, "_", "nz"), nz_mean)
            dzi = get(Nuisances, string(name, "_", "dz"), 0.0)
            zs = zs_mean .+ dzi  # Opposite sign in KiDS
            tracer = NumberCountsTracer(cosmology, zs, nz;
                                        b=b)
        elseif t_type == "e"
            nzs = files[string("nz_", name)]
            nzs = [nzs[i,:] for i in 1:size(nzs,1)]
            zs_mean, nz_mean = nzs[1], nzs[2]

            mb = get(Nuisances, string(name, "_", "mb"), 0.0)
            IA_params = [get(Nuisances, "A_IA", 0.0),
                         get(Nuisances, "alpha_IA", 0.0)]
            nz = get(Nuisances, string(name, "_", "nz"), nz_mean)
            dzi = get(Nuisances, string(name, "_", "dz"), 0.0)
            zs = zs_mean .+ dzi  # Opposite sign in KiDS
            tracer = WeakLensingTracer(cosmology, zs, nz;
                                       mb=mb, IA_params=IA_params)
            
        elseif t_type == "k"
            tracer = CMBLensingTracer(cosmology)

        else
            print("Not implemented")
            tracer = nothing
        end
        merge!(tracers, Dict(name => tracer))
    end

    npairs = length(pairs)
    total_len = last(idx)
    cls = zeros(cosmology.settings.cosmo_type, total_len)
    @inbounds Threads.@threads :static for i in 1:npairs
        name1, name2 = pairs[i]
        ls = files[string("ls_", name1, "_", name2)]
        tracer1 = tracers[name1]
        tracer2 = tracers[name2]
        cls[idx[i]+1:idx[i+1]] = angularCℓs(cosmology, tracer1, tracer2, ls)
    end
    
    return cls
end

TTheory (generic function with 1 method)

In [12]:
using ForwardDiff
function diff_cls(Wb)
    cosmology = LimberJack.Cosmology(0.3, Wb, 0.67, 0.96, 0.81,
                                     tk_mode="EisHu", Pk_mode="Halofit")
    cls = TTheory(cosmology, tracers_names, pairs,
                 idx, files; Nuisances=nuisances)
    return cls
end

function numd_cls(Wb)
    cosmology1 = LimberJack.Cosmology(0.3, Wb, 0.67, 0.96, 0.81,
                                     tk_mode="EisHu", Pk_mode="Halofit")
    cosmology2 = LimberJack.Cosmology(0.3, Wb-0.0001, 0.67, 0.96, 0.81,
                                     tk_mode="EisHu", Pk_mode="Halofit")
    cls1 = TTheory(cosmology1, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
    cls2 = TTheory(cosmology2, tracers_names, pairs,
                  idx, files; Nuisances=nuisances)
    return (cls1 .- cls2) ./ 0.0001
end


function diff_cls_nuisance(dz)
    nuisances = Dict(#"b0" => 2.0,
                     "DESgc__1_0_dz"=> dz)
    cosmology = LimberJack.Cosmology(0.3, 0.05, 0.67, 0.96, 0.81,
                                     tk_mode="EisHu", Pk_mode="Halofit")
    cls = TTheory(cosmology, tracers_names, pairs,
                 idx, files; Nuisances=nuisances)
    return cls
end

function numd_cls_nuisance(dz)
    nuisances1 = Dict(#"b0" => 2.0,
                      "DESgc__1_0_dz"=> dz)
    nuisances2 = Dict(#"b0" => 2.0,
                      "DESgc__1_0_dz"=> dz.-0.005)
    cosmology = LimberJack.Cosmology(0.3, 0.05, 0.67, 0.96, 0.81,
                                     tk_mode="EisHu", Pk_mode="Halofit")
    cls1 = TTheory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances1)
    cls2 = TTheory(cosmology, tracers_names, pairs,
                  idx, files; Nuisances=nuisances2)
    return (cls1 .- cls2) ./ 0.0005
end

numd_cls_nuisance (generic function with 1 method)

In [13]:
function dzz(p::T)::Array{T,1} where T<:Real
    cosmo = LimberJack.Cosmology(0.3, 0.05, 0.67, 0.96, 0.81,
                                 tk_mode="EisHu", Pk_mode="Halofit",
                                 nz=300)
    cosmo.settings.cosmo_type = typeof(p)
    z = Vector(range(0., stop=2., length=256)) .- p
    nz = Vector(@. exp(-0.5*((z-0.5)/0.05)^2))
    tg = NumberCountsTracer(cosmo, z, nz; b=1)
    ℓs = [10.0, 30.0, 100.0, 300.0]
    Cℓ_gg = angularCℓs(cosmo, tg, tg, ℓs) 
    return Cℓ_gg
end

dzz (generic function with 1 method)

In [14]:
d = 0.00005

5.0e-5

In [15]:
dz_autodiff = ForwardDiff.derivative(dzz, -0.1)

LoadError: InterruptException:

In [16]:
dz_anal = (dzz(-0.1+d)-dzz(-0.1-d))/2d

4-element Vector{Float64}:
 -1.12379536306871e-6
 -1.0490195590473711e-5
  7.36721103335858e-6
 -9.817539999908508e-7

In [17]:
abs.(dz_autodiff ./ dz_anal .-1)

LoadError: UndefVarError: dz_autodiff not defined

In [18]:
#mean(abs.(numd_cls(0.05)./ForwardDiff.derivative(diff_cls, 0.05) .-1))

In [83]:
dz=0.01
nuisances1 = Dict(#"b0" => 2.0,
                      "DESgc__1_0_dz"=> dz)
nuisances2 = Dict(#"b0" => 2.0,
                  "DESgc__1_0_dz"=> dz.-0.005)
cosmology = LimberJack.Cosmology(0.3, 0.05, 0.67, 0.96, 0.81,
                                 tk_mode="EisHu", Pk_mode="Halofit")
cls1 = TTheory(cosmology, tracers_names, pairs,
              idx, files; Nuisances=nuisances1)
cls2 = TTheory(cosmology, tracers_names, pairs,
              idx, files; Nuisances=nuisances2)
(cls1 .- cls2) ./ 0.0005

475-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.0019234222521184719
 -0.0006534361028570078
 -0.0003303224617137253
 -0.00011796924193214376
 -0.00011472270205373865
 -4.912254074234995e-5
 -7.518756365624854e-5
 -5.297606257727153e-5
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [85]:
numd_cls_nuisance(0.01)

475-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.0019234222521184719
 -0.0006534361028570078
 -0.0003303224617137253
 -0.00011796924193214376
 -0.00011472270205373865
 -4.912254074234995e-5
 -7.518756365624854e-5
 -5.297606257727153e-5
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [81]:
mean(abs.(numd_cls_nuisance(0.1)./ForwardDiff.derivative(diff_cls_nuisance, 0.1) .-1))

NaN

In [21]:
nume = numd_cls_nuisance(-0.01)
anal = ForwardDiff.derivative(diff_cls_nuisance, -0.01)
n = length(nume)
comp = zeros(Float64, n)

for i in 1:n 
    if (nume[i] != 0)
        comp[i] = @.(abs(anal[i]/nume[i] - 1))
    end
end

diff = []
for i in 1:n 
    if (nume[i] != 0)
        push!(diff, @.(abs(nume[i]-anal[i])))
    end
end
println(mean(diff))

4.72991988203448e-7


## Sample Prior

In [58]:
function cconditional(old_X, new_X,
                     mean, new_mean, latent_gp, 
                     cov_fn; kwargs...)
    N = length(new_X)
    Z = [new_X; old_X]
    K = cov_fn(Z; kwargs...)
    Koo = K[(N+1):end, (N+1):end]    
    Kno = K[1:N, (N+1):end]          
    
    Koo_inv = inv(Koo)
    C = Kno * Koo_inv
    return new_mean .+ C * (mean .- latent_gp)
end


cconditional (generic function with 1 method)

In [59]:
function make_theory(Ωm, v;
                      tracers_names=tracers_names,
                      pairs=pairs,
                      idx=idx,
                      cov_tot=cov_tot, 
                      files=files,
                      fid_cosmo=fid_cosmo,
                      latent_x=latent_x,
                      x=x)
    

    Ωm = pars[26]
    Ωb = pars[1]
    h = pars[4]
    ns = pars[3]
    s8 = 0.811

    eta = 0.1
    l = 3

    latent_N = length(latent_x)
    
    mu = fid_cosmo.Dz(vec(latent_x))
    mu_dense = fid_cosmo.Dz(vec(x))
    K = sqexp_cov_fn(latent_x; eta=eta, l=l)
    latent_gp = latent_GP(mu, v, K)
    gp = cconditional(latent_x, x, 
                      mu, mu_dense, latent_gp, 
                      sqexp_cov_fn; eta=eta, l=l)
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit";
                                     custom_Dz=[x, gp])
    
    cls = Theory(cosmology, tracers_names, pairs,
                    idx, files; Nuisances=nuisances)
    
    fs8s = fs8(cosmology, x)
    
    return gp, fs8s, cls
end;

In [60]:
N_samples = 100
cls_preds = zeros(N_samples, length(cls_data))
gps_preds = zeros(N_samples, length(x))
fs8s_preds = zeros(N_samples, length(x))
for i in 1:N_samples
    Ωm = rand(Uniform(0.1, 0.6))
    v = rand(MvNormal(zeros(length(latent_x)), ones(length(latent_x))))
    gps_preds[i,:], fs8s_preds[i,:], cls_preds[i,:] = make_theory(Ωm, v)
end

In [61]:
cls_preds_m, cls_preds_s = mean(cls_preds, dims=1), std(cls_preds, dims=1);
fs8s_preds_m, fs8s_preds_s = mean(fs8s_preds, dims=1), std(fs8s_preds, dims=1);
gps_preds_m, gps_preds_s = mean(gps_preds, dims=1), std(gps_preds, dims=1);

In [62]:
npzwrite("fs8_ploting.npz", Dict("cls_m" => cls_preds_m,
                                 "cls_s" => cls_preds_s, 
                                 "fs8s_m" => fs8s_preds_m,
                                 "fs8s_s" => fs8s_preds_s,
                                 "gps_m" => gps_preds_m,
                                 "gps_s" => gps_preds_s)) 

## Sample Posterior

In [46]:
@model function model(data_vector;
                      tracers_names=tracers_names,
                      pairs=pairs,
                      idx=idx,
                      cov_tot=cov_tot, 
                      files=files,
                      fid_cosmo=fid_cosmo,
                      latent_x=latent_x,
                      x=x)

    #KiDS priors
    Ωm ~ Uniform(0.2, 0.6)
    Ωb = 0.05 # ~ Uniform(0.028, 0.065)
    h = 0.67 #~ Uniform(0.64, 0.82)
    ns = 0.96 #~ Uniform(0.84, 1.1)
    s8 = 0.811
    
    eta = 0.05
    l = 1
    latent_N = length(latent_x)
    v ~ filldist(truncated(Normal(0, 1), -3, 3), latent_N)
    
    mu = fid_cosmo.Dz(vec(latent_x))
    K = sqexp_cov_fn(latent_x; eta=eta, l=l)
    latent_gp = latent_GP(mu, v, K)
    gp = conditional(latent_x, x, latent_gp, sqexp_cov_fn;
                      eta=eta, l=l)

    dmu =  growth_rate(fid_cosmo, vec(latent_x)) ./ (1 .+ latent_x)
    dK = sqexp_cov_grad(latent_x; eta=eta, l=l)
    latent_dgp = latent_GP(dmu, v, dK)
    dgp = conditional(latent_x, x, latent_dgp, sqexp_cov_grad;
                      eta=eta, l=l)
    
    cosmology = Cosmology(Ωm, Ωb, h, ns, s8,
                          tk_mode="EisHu",
                          Pk_mode="Halofit", 
                          custom_Dz=[x, gp, dgp])
    
    cls = Theory(cosmology, tracers_names, pairs,
                    idx, files; Nuisances=nuisances)
    
    fs8s = fs8(cosmology, fs8_zs)
    theory = [fs8s; cls]
    
    data_vector ~ MvNormal(theory, cov_tot)
end;

In [47]:
chain3 = sample(model(data_vector), NUTS(5, 0.1; init_ϵ=0.03), iterations,
                progress=true; save_state=true);

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
Sampling:   4%|█▌                                       |  ETA: 0:01:18┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
Sampling:   5%|██▎                                      |  ETA: 0:01:23┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/

Sampling:  51%|████████████████████▉                    |  ETA: 0:00:38┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
Sampling:  53%|█████████████████████▋                   |  ETA: 0:00:37┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
Sampling:  54%|██████████████████████▍                  |  ETA: 0:00:35┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
Sampling:  56%|███████████████████████▏                 |  ETA: 0:00:34┌ Warning: The current proposal will be rejected due to 

In [40]:
chain1 = sample(model(data_vector), MH(), iterations,
            nadapt=nadapts, progress=true; save_state=true);

LoadError: UndefVarError: nadapts not defined